In [1]:
# Importing stock ml libraries
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
from data_preprocessing import ANOMALY_LABELS, ANOMALY_SUBCATEGORIES, load_data
from custom_dataset import CustomDataset
from model import SequenceClassificationModel
from metrics import custom_classification_report
from losses import loss

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Defining some key variables that will be used later on in the training
MODEL_NAME = "AeroBOT"
MODEL_DIRECTORY = "model_save"


# ABBREVIATION = True
ABBREVIATION = False


LOSS_TYPE = 'BCE'
# LOSS_TYPE = 'BinaryFocal'

BALANCED = False
# BALANCED = True

# LAYERS_TO_UNFREEZE = None
LAYERS_TO_UNFREEZE = [8, 9, 10, 11]

ENCODER_NAME = 'bert-base-uncased'
# ENCODER_NAME = 'NASA-AIML/MIKA_SafeAeroBERT'
# ENCODER_NAME = 'allenai/longformer-base-4096'

IS_SUBCATEGORY = False


if MODEL_NAME == 'AeroBOT':
    MAX_LEN = 200
elif ENCODER_NAME == 'allenai/longformer-base-4096':
    MAX_LEN = 1024
else:
    MAX_LEN = 512

TRAIN_EFFECTIVE_BATCH_SIZE = 32 # 32 Effective size for NASA
TRAIN_BATCH_SIZE = 32
ACCUMULATION_STEPS = TRAIN_EFFECTIVE_BATCH_SIZE / TRAIN_BATCH_SIZE
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA

drop the NaN values in Anomaly?

In [6]:
if IS_SUBCATEGORY:
    labels = ANOMALY_SUBCATEGORIES
else:
    labels = ANOMALY_LABELS
    
if ABBREVIATION:
    train_df = load_data("./data/train_data_final.pkl", labels, pp_path="./data/train_data_processed2.pkl")
    test_df = load_data("./data/test_data_final.pkl", labels, pp_path="./data/test_data_processed2.pkl")
else: 
    train_df = load_data("./data/train_data_final.pkl", labels)
    test_df = load_data("./data/test_data_final.pkl", labels)
train_df

,text,labels
ACN,,
1163382,I was the pilot flying performing the takeoff....,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
893734,We had 6 shipments of dry ice for the flight; ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
991883,I have seen a lot of mistakes on every flight ...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1590076,It was my first time flying into KEUG and I wa...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1715282,I am writing this report to bring attention to...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
622204,WE WERE ENRTE IN LNAV AT FL310; 30 MI N OF ATL...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
622205,CLRED BY TWR CTL TO CROSS RWY 8R/26L AT TXWY E...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
661202,WHILE WORKING NUMEROUS CVG AND CMH DEPS AT A C...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
test_df

,text,labels
ACN,,
1014798,Flying into SLC on the DELTA THREE RNAV arriva...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1806744,ORD was on a very busy east flow arrival push....,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1044902,B737-800 was vectored to an ILS Runway 16L app...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1764093,We were on a 6 mile final when tower cleared a...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
1786435,During Climb we Leveled at 17;000 departure sw...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
1310569,FO was flying a visual approach to runway 26 i...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1482118,While assembling a GE C2 transfer gearbox; I n...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1565471,Nearing the end of a hot; bumpy four-hour IFR ...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
num_labels = len(test_df.labels.iloc[0])
model = SequenceClassificationModel(ENCODER_NAME, num_labels=num_labels, model_name=MODEL_NAME)
model.set_trainable_layers(LAYERS_TO_UNFREEZE)
print(model.to(device))

if ABBREVIATION:
    model.model_name += '_Abbreviated'


SequenceClassificationModel(
  (l1): AeroBOTSequenceClassification(
    (encoder): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_

In [9]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

tokenizer = model.tokenizer()
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (96986, 2)
TEST Dataset: (10805, 2)


In [10]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [11]:
loss_fn = loss(model, LOSS_TYPE, BALANCED, training_set, training_loader, device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {
    "Custom Classification Report": lambda y_true, y_pred: custom_classification_report(y_true, y_pred, labels),
    "Optimization Metric": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=0)
}


In [12]:
model.model_name

'bert-base-uncased_Unfrozen[8, 9, 10, 11]_BCE'

In [13]:
from training import TrainingHandler
trainer = TrainingHandler(model, optimizer, loss_fn, device, labels, metrics_dict, directory=MODEL_DIRECTORY, base_path=MODEL_NAME)
trainer.train(training_loader, testing_loader, save=True, epochs=EPOCHS)

No saved model found.
Resuming training from epoch 0
Training - Epoch: 1, Batch: 6062 [60620/96986], Time: 492s 54ms/step, Loss: 0.099205
Model saved at model_save/AeroBOT_epoch_0.pth
Train Results:
Average Loss: 0.2302
Optimization Metric: 0.4112

Custom Classification Report:
  Deviation / Discrepancy - Procedural:  binary_accuracy: 0.7208, precision: 0.7281, recall: 0.8553, f1-score: 0.7866, support: 58337.0000
  Aircraft Equipment:                    binary_accuracy: 0.8654, precision: 0.8455, recall: 0.8025, f1-score: 0.8234, support: 37932.0000
  Conflict:                              binary_accuracy: 0.8937, precision: 0.8194, recall: 0.5200, f1-score: 0.6363, support: 17342.0000
  Inflight Event / Encounter:            binary_accuracy: 0.8724, precision: 0.7831, recall: 0.3881, f1-score: 0.5190, support: 17203.0000
  ATC Issue:                             binary_accuracy: 0.8864, precision: 0.7636, recall: 0.4356, f1-score: 0.5547, support: 15751.0000
  Deviation - Altitude:   

In [14]:
from evaluation import EvaluationHandler
evaluator = EvaluationHandler(model, device, labels, metrics_dict)
loss, metrics_results, thresholds = evaluator.evaluate(testing_loader, loss_fn, optimize=True)
thresholds

Evaluation - Batch: 338 [ 7098/10805], Time: 32s 62ms/step, Loss: 0.133948
Optimizing Thresholds
Evaluation Results:
Average Loss: 0.1637
Optimization Metric: 0.6638

Custom Classification Report:
  Deviation / Discrepancy - Procedural:  binary_accuracy: 0.7596, precision: 0.7499, recall: 0.8862, f1-score: 0.8123, support: 6343.0000
  Aircraft Equipment:                    binary_accuracy: 0.9050, precision: 0.8837, recall: 0.8798, f1-score: 0.8817, support: 4351.0000
  Conflict:                              binary_accuracy: 0.9451, precision: 0.8521, recall: 0.8281, f1-score: 0.8399, support: 1879.0000
  Inflight Event / Encounter:            binary_accuracy: 0.8854, precision: 0.7400, recall: 0.7540, f1-score: 0.7469, support: 2423.0000
  ATC Issue:                             binary_accuracy: 0.8977, precision: 0.7769, recall: 0.7883, f1-score: 0.7825, support: 2522.0000
  Deviation - Altitude:                  binary_accuracy: 0.9407, precision: 0.6613, recall: 0.6840, f1-score: 0.

[0.34,
 0.49,
 0.67,
 0.31,
 0.46,
 0.32,
 0.34,
 0.33,
 0.48,
 0.44,
 0.23,
 0.2,
 0.4,
 0.05]

In [15]:
from model_management import ModelManager
ModelManager.save_results(MODEL_NAME, metrics_results, thresholds, "results.csv")